# T5. REFERENCE LEVELS

In this notebook, we extract the reference levels from the t1_semantic segmentation/ t2_instance segmentation

## LIBRARIES

In [123]:
#IMPORT PACKAGES
from rdflib import Graph, URIRef
import os.path
import importlib
from pathlib import Path
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d
import uuid    
import pye57 
import ifcopenshell
import ifcopenshell.geom as geom
import ifcopenshell.util
from ifcopenshell.util.selector import Selector
import multiprocessing
import random as rd
import pandas as pd
# from tabulate import tabulate
import cv2
import laspy
import json
from scipy.spatial.transform import Rotation   
import copy
import geomapi
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl

#import utils
from context import utils
import utils as utl
import utils.t1_utils as t1


In [124]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [125]:
%autoreload 2

## INPUTS

In [126]:
name='beton_labels'

path=Path(os.getcwd()).parents[2]/'data'
pcd_input_path=os.path.join(path,f'{name}.laz')
class_file=path/'_classes.json'

name=name.split('_')[0]
json_output_path=os.path.join(path,f'{name}_levels.json') 
geometry_output_path= os.path.join(path,f'{name}_levels.obj') # these are the bounding surfaces of the reference levels (optional)

#bimfolder
# bimFolder=os.mkdir(path/name/'BIM')
graphPath=str(path/f'{name}Graph.ttl')

Import Classes

In [127]:
# Read the JSON file
with open(class_file, 'r') as file:
    json_data = json.load(file)

# Create a dictionary
class_dict = {
    'classes': json_data['classes'],
    'default': json_data['default'],
    'type': json_data['type'],
    'format': json_data['format'],
    'created_with': json_data['created_with']
}
print(class_dict)

{'classes': [{'name': 'Unassigned', 'id': 255, 'temp_id': -1, 'color': '#9da2ab'}, {'name': 'Floors', 'id': 0, 'temp_id': 0, 'color': '#03c2fc'}, {'name': 'Ceilings', 'id': 1, 'temp_id': 1, 'color': '#e81416'}, {'name': 'Walls', 'id': 2, 'temp_id': 2, 'color': '#ffa500'}, {'name': 'Columns', 'id': 3, 'temp_id': 3, 'color': '#faeb36'}, {'name': 'Doors', 'id': 4, 'temp_id': 4, 'color': '#79c314'}, {'name': 'Windows', 'id': 5, 'temp_id': 5, 'color': '#4b369d'}], 'default': 255, 'type': 'semantic_segmentation', 'format': 'kitti', 'created_with': {'name': 'Saiga', 'version': '1.0.1'}}


Import PCD

In [128]:
laz=laspy.read(pcd_input_path)

In [129]:
# pcdNodes=[]
# #split pcd per object
# for i in np.unique(laz['classes']):
#     idx=np.where(laz['classes']==i)
#     points=laz.xyz[idx]
#     # colors=np.array([laz.red[idx],laz.green[idx],laz.blue[idx]])
#     object_labels=laz['objects'][idx]

#     class_obj=next((class_obj for class_obj in json_data['classes'] if float(class_obj['id']) ==i), json_data['classes'][0])
#     class_name=class_obj['name']

#     # pcd.colors=o3d.utility.Vector3dVector(colors)
#     for j in np.unique(object_labels):
        
#         new_points=points[np.where(object_labels==j)]
#         if new_points.shape[0]>100:
#             pcd=o3d.geometry.PointCloud()
#             pcd.points=o3d.utility.Vector3dVector(new_points)

#             pcdNodes.append(PointCloudNode(resource=pcd,
#                                         class_id=i,
#                                         object_id=j,
#                                         color=ut.random_color(),
#                                             name=class_name+f'_{j}'))

# print(f'{len(pcdNodes)} pcdNodes created!')

import Graph

In [130]:
graph=Graph().parse(graphPath)
pcdNodes=tl.graph_to_nodes(graph)

match point clouds with graph

In [131]:
for n in pcdNodes: # this is 6 times slower than the code above !!
    idx=np.where((laz['classes']==n.class_id) & (laz['objects']==n.object_id))
    pcd=o3d.geometry.PointCloud()
    pcd.points=o3d.utility.Vector3dVector(laz.xyz[idx])
    n.resource=pcd

In [132]:
{key:value for key, value in pcdNodes[1].__dict__.items() if not key.startswith('__') and not callable(key)}              

{'_e57Index': 0,
 'pointCount': 6778,
 'e57XmlPath': None,
 '_cartesianBounds': array([1.03781368e+05, 1.03784172e+05, 1.94669672e+05, 1.94672953e+05,
        9.32700000e+00, 9.41400000e+00]),
 '_orientedBounds': array([[1.03782598e+05, 1.94669406e+05, 9.29465039e+00],
        [1.03784583e+05, 1.94672383e+05, 9.33818767e+00],
        [1.03781107e+05, 1.94670399e+05, 9.30827883e+00],
        [1.03782598e+05, 1.94669404e+05, 9.39478446e+00],
        [1.03783093e+05, 1.94673375e+05, 9.45195018e+00],
        [1.03781107e+05, 1.94670398e+05, 9.40841290e+00],
        [1.03784583e+05, 1.94672382e+05, 9.43832174e+00],
        [1.03783093e+05, 1.94673377e+05, 9.35181611e+00]]),
 '_orientedBoundingBox': None,
 '_subject': rdflib.term.URIRef('file:///0_Floors_1'),
 '_graph': <Graph identifier=N3c194acddfd44077b9f69746fabd4fae (<class 'rdflib.graph.Graph'>)>,
 '_graphPath': None,
 '_path': None,
 '_name': '0_Floors_1',
 '_timestamp': None,
 '_resource': PointCloud with 6778 points.,
 '_cartesianTr

In [133]:
# joined_pcd=gmu.join_geometries([n.resource.paint_uniform_color(ut.literal_to_array(n.color)) for n in pcdNodes if n.resource is not None])
# o3d.visualization.draw_geometries([joined_pcd])

## PROCESSING

Group floor point clouds

In [134]:
#retrieve floor points
floorNodes=[n for n in pcdNodes if n.class_id ==0]
print(f'{len(floorNodes)} floorNodes detected!')
for n in floorNodes:
    print(f' {n.name} with {n.get_oriented_bounding_box()}')


9 floorNodes detected!
 0_Floors_1 with OrientedBoundingBox: center: (103783, 194671, 9.3733), extent: 3.57975, 1.79201, 0.100157)
 0_Floors_2 with OrientedBoundingBox: center: (103778, 194657, 9.15455), extent: 16.7689, 2.42513, 0.118288)
 0_Floors_3 with OrientedBoundingBox: center: (103777, 194666, 9.19714), extent: 7.45786, 1.36485, 0.0926567)
 0_Floors_4 with OrientedBoundingBox: center: (103788, 194655, 9.14621), extent: 9.95776, 1.39346, 0.230278)
 0_Floors_5 with OrientedBoundingBox: center: (103793, 194660, 9.22532), extent: 4.40927, 1.36758, 0.082956)
 0_Floors_6 with OrientedBoundingBox: center: (103797, 194663, 9.23211), extent: 4.50235, 1.15998, 0.0803558)
 0_Floors_7 with OrientedBoundingBox: center: (103793, 194670, 9.2833), extent: 17.5246, 6.25761, 0.345612)
 0_Floors_8 with OrientedBoundingBox: center: (103786, 194664, 9.11211), extent: 19.7788, 14.7034, 0.260121)
 0_Floors_9 with OrientedBoundingBox: center: (103788, 194671, 9.11058), extent: 6.64896, 5.22037, 0.1249

In [135]:
# for now, put all floorNodes in a single group (1 reference level)
groupedFloorNodes=[floorNodes]
print(groupedFloorNodes)


[[<geomapi.nodes.pointcloudnode.PointCloudNode object at 0x00000234CF9F6B90>, <geomapi.nodes.pointcloudnode.PointCloudNode object at 0x00000234C0738C10>, <geomapi.nodes.pointcloudnode.PointCloudNode object at 0x00000234D32490F0>, <geomapi.nodes.pointcloudnode.PointCloudNode object at 0x00000234C0738B80>, <geomapi.nodes.pointcloudnode.PointCloudNode object at 0x00000234C0738940>, <geomapi.nodes.pointcloudnode.PointCloudNode object at 0x00000234C07388B0>, <geomapi.nodes.pointcloudnode.PointCloudNode object at 0x00000234C0738BE0>, <geomapi.nodes.pointcloudnode.PointCloudNode object at 0x00000234C0738A00>, <geomapi.nodes.pointcloudnode.PointCloudNode object at 0x00000234C0738880>]]


Compute reference Planes

In [137]:
#compute height underneight all floor points of a level (according to the lowest slab)
referenceNodes= []

for i,nodes in enumerate(groupedFloorNodes):
    #create sessionNode
    name='level_'+str(i)+'0'
    referenceNode=SessionNode(linkedNodes=nodes,
                                name=name,
                                subject=name)
    
    #determine height -> note that this can be negative
    weights=[float(len(np.asarray(n.resource.points))) for n in nodes]
    heights= [float(n.cartesianTransform[2,3]) for n in nodes]
    weighted_height= np.average(heights, weights=weights)

    #compute plane from cornerpoints orientedbounding box
    vertices=np.array([np.hstack((referenceNode.orientedBounds[0][0:2],weighted_height)),
                       np.hstack((referenceNode.orientedBounds[1][0:2],weighted_height)),
                       np.hstack((referenceNode.orientedBounds[2][0:2],weighted_height)),
                       np.hstack((referenceNode.orientedBounds[4][0:2],weighted_height))])#,
    vertices=o3d.cpu.pybind.utility.Vector3dVector(vertices)
    triangles=o3d.cpu.pybind.utility.Vector3iVector(np.array([[0,1,2],[2,1,3]]))
    plane=o3d.geometry.TriangleMesh(vertices,triangles)

    #assign information to referenceNode
    referenceNode.plane=plane
    referenceNode.height=weighted_height
    
    referenceNodes.append(referenceNode)
    print(f' Level {referenceNode.name} created at height {referenceNode.height}')
# print(f'{len(referenceNodes)} referenceNodes created!')

 Level level_00 created at height 9.149400542324619


In [138]:
# joined_pcd=gmu.join_geometries([n.resource.paint_uniform_color(n.color) for n in pcdNodes if n.resource is not None])
# o3d.visualization.draw_geometries([joined_pcd,referenceNodes[0].plane])

In [139]:
{key:value for key, value in referenceNode.__dict__.items() if not key.startswith('__') and not callable(key)}              

{'_linkedNodes': [<geomapi.nodes.pointcloudnode.PointCloudNode at 0x234cf9f6b90>,
 '_linkedSubjects': [rdflib.term.URIRef('file:///0_Floors_1'),
  rdflib.term.URIRef('file:///0_Floors_2'),
  rdflib.term.URIRef('file:///0_Floors_3'),
  rdflib.term.URIRef('file:///0_Floors_4'),
  rdflib.term.URIRef('file:///0_Floors_5'),
  rdflib.term.URIRef('file:///0_Floors_6'),
  rdflib.term.URIRef('file:///0_Floors_7'),
  rdflib.term.URIRef('file:///0_Floors_8'),
  rdflib.term.URIRef('file:///0_Floors_9')],
 '_subject': rdflib.term.URIRef('file:///level_00'),
 '_graph': None,
 '_graphPath': None,
 '_path': None,
 '_name': 'level_00',
 '_cartesianBounds': array([1.03771625e+05, 1.03801741e+05, 1.94649782e+05, 1.94678155e+05,
        8.94960644e+00, 9.58402653e+00]),
 '_orientedBounds': array([[1.03789054e+05, 1.94646344e+05, 8.72455075e+00],
        [1.03802766e+05, 1.94669200e+05, 8.87839540e+00],
        [1.03769417e+05, 1.94658124e+05, 8.85199679e+00],
        [1.03789055e+05, 1.94646339e+05, 9.333

Compute Json information

In [140]:
for n in referenceNodes:
    n.box=pcd.get_oriented_bounding_box()
    n.box.color=[1,0,0]
    
    rotation_matrix=copy.deepcopy(n.box.R) #! can these angles be negative?
    r =  Rotation.from_matrix(np.asarray(rotation_matrix))
    n.rotations = r.as_euler("zyx",degrees=True)

## EXPORT

geometry

In [141]:
joined_references=gmu.join_geometries([n.resource for n in referenceNodes])
success=o3d.io.write_triangle_mesh(filename=geometry_output_path, mesh=joined_references)
print(f' Saving joint references : {success}')
    

 Saving joint references : True


json with reference heights

In [142]:
#declare json
json_data = {
        "filename": ut.get_filename(geometry_output_path),
        "objects": []
    }

In [143]:
#fill json
for n in referenceNodes:
    obj = {
            "name": 'level_'+n.name,
            "centroid": {
                "x": n.box.center[0],
                "y": n.box.center[1],
                "z": n.height
            },
            "dimensions": {
                "length": n.box.extent[0],
                "width": n.box.extent[1],
                "height": n.height
            },
            "rotations": {
                "x": 0,
                "y": 0,
                "z": n.rotations[0]
            }
            }
    json_data["objects"].append(obj)

In [144]:
#write this information to the 3D detection json
with open(json_output_path, "w") as json_file:
    json.dump(json_data, json_file, indent=4)
print("JSON data written to file:", json_output_path)

JSON data written to file: c:\Users\Maarten\OneDrive - KU Leuven\2024-05 CVPR scan-to-BIM challenge\data\beton_levels.json


save Graph

In [145]:
new_graph=tl.nodes_to_graph(referenceNodes,
                  graphPath=graphPath)
graph=graph+new_graph
graph.serialize(graphPath)

<Graph identifier=N3f2ef880db314425aedd927d8be3b4fa (<class 'rdflib.graph.Graph'>)>